# BentoML Example: spaCy named entity recognizer  


**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


Make sure to __use GPU runtime when running this notebook in Google Colab__, you can set it in top menu: `Runtime > Change Runtime Type > Hardware accelerator`.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=spacy&ea=spacy-ner&dt=spacy-ner)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
!pip install -q bentoml spacy>=2.3.0

You should consider upgrading via the '/usr/local/anaconda3/envs/dev-py3/bin/python -m pip install --upgrade pip' command.


In [2]:
!python3 -m spacy download en_core_web_sm

You should consider upgrading via the '/usr/local/anaconda3/envs/dev-py3/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
import en_core_web_sm

nlp = en_core_web_sm.load()

# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [4]:
# training data
TRAIN_DATA = [
              ("Walmart is a leading e-commerce company", {"entities": [(0, 7, "ORG")]}),
              ("I reached Chennai yesterday.", {"entities": [(19, 28, "GPE")]}),
              ("I recently ordered a book from Amazon", {"entities": [(24,32, "ORG")]}),
              ("I was driving a BMW", {"entities": [(16,19, "PRODUCT")]}),
              ("I ordered this from ShopClues", {"entities": [(20,29, "ORG")]}),
              ("Fridge can be ordered in Amazon ", {"entities": [(0,6, "PRODUCT")]}),
              ("I bought a new Washer", {"entities": [(16,22, "PRODUCT")]}),
              ("I bought a old table", {"entities": [(16,21, "PRODUCT")]}),
              ("I bought a fancy dress", {"entities": [(18,23, "PRODUCT")]}),
              ("I rented a camera", {"entities": [(12,18, "PRODUCT")]}),
              ("I rented a tent for our trip", {"entities": [(12,16, "PRODUCT")]}),
              ("I rented a screwdriver from our neighbour", {"entities": [(12,22, "PRODUCT")]}),
              ("I repaired my computer", {"entities": [(15,23, "PRODUCT")]}),
              ("I got my clock fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("I got my truck fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("Flipkart started it's journey from zero", {"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Max", {"entities": [(24,27, "ORG")]}),
              ("Flipkart is recognized as leader in market",{"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Swiggy", {"entities": [(24,29, "ORG")]})
              ]

In [5]:
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])
    
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [6]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(300):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

Losses {'ner': 6.793484245426953}
Losses {'ner': 11.22245792252943}
Losses {'ner': 12.999693839577958}
Losses {'ner': 16.720011212630197}
Losses {'ner': 22.314620407627444}
Losses {'ner': 3.4436206984100863}
Losses {'ner': 3.6815590899204835}
Losses {'ner': 4.65945196559187}
Losses {'ner': 12.351311054517282}
Losses {'ner': 17.30548254583846}
Losses {'ner': 4.034456621389836}
Losses {'ner': 6.063596666761441}
Losses {'ner': 13.533002676878823}
Losses {'ner': 15.843369366608385}
Losses {'ner': 19.643523378304963}
Losses {'ner': 5.578153549526178}
Losses {'ner': 10.864336172882759}
Losses {'ner': 13.35362255142536}
Losses {'ner': 18.992132167939417}
Losses {'ner': 19.441392632019415}
Losses {'ner': 0.2953134557210433}
Losses {'ner': 5.536041476621904}
Losses {'ner': 10.480906741459279}
Losses {'ner': 13.397003056385387}
Losses {'ner': 13.686604858527375}
Losses {'ner': 6.776049214517116}
Losses {'ner': 7.188584149613234}
Losses {'ner': 9.517494688889201}
Losses {'ner': 11.517572496379216

Losses {'ner': 1.5711554509654144}
Losses {'ner': 1.5711554968311214}
Losses {'ner': 1.5711583902582278}
Losses {'ner': 1.7898388005469907}
Losses {'ner': 0.03770482944020448}
Losses {'ner': 0.0613161028831446}
Losses {'ner': 0.06168050442018851}
Losses {'ner': 0.061680962781734586}
Losses {'ner': 0.06168098202901408}
Losses {'ner': 0.09786139128875737}
Losses {'ner': 0.09798271001253382}
Losses {'ner': 0.12889271964240284}
Losses {'ner': 0.12889465021494678}
Losses {'ner': 0.12889531946185048}
Losses {'ner': 0.0698594831392243}
Losses {'ner': 0.0698595017255927}
Losses {'ner': 0.09651936576350088}
Losses {'ner': 0.09895458248653888}
Losses {'ner': 0.09895476916511753}
Losses {'ner': 0.0013747215123573187}
Losses {'ner': 0.0013748601736074913}
Losses {'ner': 0.001374881548786951}
Losses {'ner': 0.8049622035700091}
Losses {'ner': 0.8049622043954353}
Losses {'ner': 7.499334320204554e-09}
Losses {'ner': 0.00021386599544505336}
Losses {'ner': 0.00021412208652944742}
Losses {'ner': 0.000215

Losses {'ner': 0.0010115808956123865}
Losses {'ner': 1.1748573793767141e-06}
Losses {'ner': 0.00014457556465019493}
Losses {'ner': 0.00016275733067768977}
Losses {'ner': 0.0001627654105668306}
Losses {'ner': 0.00016405569990888551}
Losses {'ner': 0.013529394503864457}
Losses {'ner': 0.01352939452612144}
Losses {'ner': 0.013531169459427757}
Losses {'ner': 0.013538839008469682}
Losses {'ner': 0.013538924606591245}
Losses {'ner': 0.0005713476612683325}
Losses {'ner': 0.0006046322449145356}
Losses {'ner': 0.0006046374480809627}
Losses {'ner': 0.0006046398985841016}
Losses {'ner': 0.000604643582234027}
Losses {'ner': 2.638922961377408e-07}
Losses {'ner': 3.4655810686038077e-07}
Losses {'ner': 0.003512165166097548}
Losses {'ner': 0.0035953377676622563}
Losses {'ner': 0.003685806869246914}
Losses {'ner': 3.0546770800478324e-09}
Losses {'ner': 0.051456391189723824}
Losses {'ner': 0.051456391217140324}
Losses {'ner': 0.05145691459139853}
Losses {'ner': 0.05146992030424576}
Losses {'ner': 2.2104

Losses {'ner': 2.3551709027499177}
Losses {'ner': 2.4131509186938915}
Losses {'ner': 2.4131509320913076}
Losses {'ner': 6.949980306432938e-08}
Losses {'ner': 1.709558194955055e-05}
Losses {'ner': 1.7095583500710086e-05}
Losses {'ner': 1.709558356096426e-05}
Losses {'ner': 1.8226540036683244e-05}
Losses {'ner': 2.893253400752029e-10}
Losses {'ner': 8.227063896113969e-07}
Losses {'ner': 8.263509148477753e-07}
Losses {'ner': 9.023429502800804e-07}
Losses {'ner': 9.023705352606662e-07}
Losses {'ner': 3.127005999687545e-09}
Losses {'ner': 0.0012191816197064948}
Losses {'ner': 0.0012197668908887267}
Losses {'ner': 0.0012198537778006456}
Losses {'ner': 0.00122087148912961}
Losses {'ner': 2.077119564411825e-08}
Losses {'ner': 0.0001979367509551632}
Losses {'ner': 0.00019795217676873248}
Losses {'ner': 0.00019795218744358544}
Losses {'ner': 0.0001979521874483864}
Losses {'ner': 3.5932821014028024e-10}
Losses {'ner': 5.176696936061751e-09}
Losses {'ner': 1.0810952283236802e-07}
Losses {'ner': 1.

Losses {'ner': 3.085836324329669e-10}
Losses {'ner': 3.107668085771983e-10}
Losses {'ner': 1.603965702631379e-05}
Losses {'ner': 1.9996941745879295}
Losses {'ner': 9.17478758657676e-11}
Losses {'ner': 2.7845944063841725e-05}
Losses {'ner': 2.788613494889536e-05}
Losses {'ner': 0.0065854447278015675}
Losses {'ner': 0.006585444727887927}
Losses {'ner': 1.1277350529923302e-10}
Losses {'ner': 0.00010379292873731906}
Losses {'ner': 0.01634067191874422}
Losses {'ner': 0.016340804535279696}
Losses {'ner': 0.01634080458894813}
Losses {'ner': 2.6172671449817737e-12}
Losses {'ner': 2.889848081985731e-07}
Losses {'ner': 2.8898509265331226e-07}
Losses {'ner': 2.889877823563402e-07}
Losses {'ner': 8.288292605624901e-07}
Losses {'ner': 2.4884691956823153e-12}
Losses {'ner': 3.299748319505103e-07}
Losses {'ner': 3.397924650212392e-07}
Losses {'ner': 3.407647591445072e-07}
Losses {'ner': 0.0009021955722980888}
Losses {'ner': 8.860202537388492e-12}
Losses {'ner': 2.3676527422351003e-07}
Losses {'ner': 

Losses {'ner': 7.03682504593486e-08}
Losses {'ner': 7.057306373992342e-08}
Losses {'ner': 7.057483651944617e-08}
Losses {'ner': 7.057483720828488e-08}
Losses {'ner': 9.366478764853089e-09}
Losses {'ner': 9.45707519749378e-09}
Losses {'ner': 9.485580851470913e-09}
Losses {'ner': 9.522746299312354e-09}
Losses {'ner': 9.887817483070293e-09}
Losses {'ner': 5.102849864577704e-13}
Losses {'ner': 5.258553980470651e-13}
Losses {'ner': 1.5353385222405392e-11}
Losses {'ner': 1.5360544509222928e-11}
Losses {'ner': 1.5567304108321303e-11}
Losses {'ner': 1.636612448434995e-13}
Losses {'ner': 8.048419586439189e-11}
Losses {'ner': 6.422997631393644e-06}
Losses {'ner': 6.422997645530055e-06}
Losses {'ner': 6.423252638074482e-06}
Losses {'ner': 8.516616776479809e-09}
Losses {'ner': 0.366044113270918}
Losses {'ner': 0.3660441163609752}
Losses {'ner': 0.366044196051853}
Losses {'ner': 0.36605011673242294}
Losses {'ner': 1.3599902851937836e-11}
Losses {'ner': 4.5503461447341575e-10}
Losses {'ner': 8.60382

Losses {'ner': 4.852479409735852e-13}
Losses {'ner': 4.018169741621716e-08}
Losses {'ner': 2.187496416217603e-05}
Losses {'ner': 2.190314051965512e-05}
Losses {'ner': 2.1903140528233246e-05}
Losses {'ner': 7.335884824374118e-13}
Losses {'ner': 3.6834449738533544e-07}
Losses {'ner': 4.2808663755789687e-07}
Losses {'ner': 4.281592848660147e-07}
Losses {'ner': 8.832839238553513e-07}
Losses {'ner': 8.674588786858036e-08}
Losses {'ner': 1.9999996099131074}
Losses {'ner': 1.9999996099203123}
Losses {'ner': 1.9999996100178685}
Losses {'ner': 1.9999996100179738}
Losses {'ner': 1.2920850825558155e-13}
Losses {'ner': 1.5415693636804232e-09}
Losses {'ner': 0.001670636032022686}
Losses {'ner': 0.0016706360320236405}
Losses {'ner': 0.0016706360327334332}
Losses {'ner': 4.469212541748844e-05}
Losses {'ner': 4.4839727733979755e-05}
Losses {'ner': 4.48445493695356e-05}
Losses {'ner': 4.4844549534173496e-05}
Losses {'ner': 4.4844554758992805e-05}
Losses {'ner': 2.3715303185484996e-14}
Losses {'ner': 3.

In [7]:
# Testing the model
doc = nlp("I was driving a Ford")
print(doc.ents)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

(Ford,)
Entities [('Ford', 'PRODUCT')]


## Create BentoService for model serving

In [8]:
%%writefile spacy_ner.py


from bentoml import BentoService, api, env, artifacts
from bentoml.frameworks.spacy import SpacyModelArtifact
from bentoml.adapters import JsonInput


@env(auto_pip_dependencies=True)
@artifacts([SpacyModelArtifact('nlp')])
class SpacyNERService(BentoService):
    @api(input=JsonInput(), batch=True)
    def predict(self, parsed_json_list):
        result = []
        for index, parsed_json in enumerate(parsed_json_list):
            doc = self.artifacts.nlp(parsed_json['text'])
            result.append([{'entity': ent.text, 'label': ent.label_} for ent in doc.ents])
        return result

Overwriting spacy_ner.py


In [9]:
from spacy_ner import SpacyNERService

svc = SpacyNERService()
svc.pack('nlp', nlp)

saved_path = svc.save()

[2020-09-22 21:16:26,556] WARNING - Importing from "bentoml.artifact.*" has been deprecated. Instead, use`bentoml.frameworks.*` and `bentoml.service.*`. e.g.:, `from bentoml.frameworks.sklearn import SklearnModelArtifact`, `from bentoml.service.artifacts import BentoServiceArtifact`, `from bentoml.service.artifacts.common import PickleArtifact`
[2020-09-22 21:16:26,614] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 21:16:27,001] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 21:16:27,789] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/setuptools/dist.py:476: UserWarning: Normalizing '0.9.0.pre+6.g4beee0a8.dirty' to '0.9.0rc0+6.g4beee0a8.dirty'
  normalized_version,
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.0rc0+6.g4beee0a8.dirty/bentoml/_version.py
set BentoML-0.9.0rc0+6.g4beee0a8.dirty/bentoml/_version.py to '0.9.0.pre+6.g4beee0a8.dirty'
[2020-09-22 21:16:31,692] INFO - BentoService bundle 'SpacyNERService:20200922211627_9930A6' saved to: /Users/bozhaoyu/bentoml/repository/SpacyNERService/20200922211627_9930A6


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [14]:
!bentoml serve SpacyNERService:latest

[2020-09-15 16:13:38,999] INFO - Getting latest version SpacyNERService:20200915161253_DC0550
[2020-09-15 16:13:38,999] INFO - Starting BentoML API server in development mode..
[2020-09-15 16:13:39,318] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-15 16:13:39,330] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+43.g53afaa73
 * Serving Flask app "SpacyNERService" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
^C


If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [23]:
!bentoml serve SpacyNERService:latest --run-with-ngrok

[2020-09-15 16:39:23,654] INFO - Getting latest version SpacyNERService:20200915161701_4475B2
[2020-09-15 16:39:23,655] INFO - Starting BentoML API server in development mode..
[2020-09-15 16:39:23,920] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-15 16:39:23,937] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+43.g53afaa73
 * Serving Flask app "SpacyNERService" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
7=ngrok by @inconshreveable                                       (Ctrl+C to quit)                                                                                S

Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.


### Send prediction requeset to the REST API server

Navigate to parent directory of the notebook(so you have reference to the `test.jpg` image), and run the following `curl` command to send the image to REST API server and get a prediction result:

```bash
curl -i \
    --request POST \
    --header "Content-Type: application/json" \
    --data "{\"text\":\"I am driving BMW\"}" \
    localhost:5000/predict
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [ ]:
!bentoml containerize SpacyNERService:latest

In [ ]:
!docker run -p 5000:5000 spacynerservice

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [25]:
from bentoml import load

service = load(saved_path)

print(service.predict([{'text': 'I am driving BMW'}]))

[2020-09-15 16:42:57,464] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+43.g53afaa73
[2020-09-15 16:42:57,465] WARNING - Module `spacy_ner` already loaded, using existing imported module.
[[{'entity': 'BMW', 'label': 'PRODUCT'}]]


## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [27]:
!bentoml run SpacyNERService:latest predict --input "{\"text\":\"I am driving BMW\"}"

[2020-09-15 16:44:08,832] INFO - Getting latest version SpacyNERService:20200915161701_4475B2
[2020-09-15 16:44:08,871] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-15 16:44:08,884] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+43.g53afaa73
[{"entity": "BMW", "label": "PRODUCT"}]


# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

